# Fiddler Evaluations SDK Quick Start

## Goal

Welcome to the **Fiddler Evaluations SDK Quick Start**! This comprehensive guide will walk you through using Fiddler's powerful evaluation framework to systematically test and evaluate your LLM applications, RAG systems, and AI agents.

### Evaluating LLM, RAG, and Agentic Applications

The Fiddler Evaluations SDK provides:

- 🧪 **Systematic Evaluation**: Run structured experiments on your AI applications
- 📊 **Built-in Evaluators**: Access to production-ready evaluators for common AI tasks
- 🔧 **Custom Evaluators**: Build custom evaluation logic for your specific use cases
- 📈 **Result Tracking**: Comprehensive experiment tracking and result analysis
- 🚀 **Scale**: Evaluate across large datasets with concurrent processing

## About Fiddler

Fiddler is the all-in-one AI Observability and Security platform for responsible AI. Monitoring and analytics capabilities provide a common language, centralized controls, and actionable insights to operationalize production predictive, generative, and agentic applications. An integral part of the platform, the Fiddler Trust Service provides quality and moderation controls for LLM applications. Powered by cost-effective, task-specific, and scalable Fiddler-developed trust models — including cloud and VPC deployments for secure environments — it delivers the fastest guardrails in the industry. Fortune 500 organizations utilize Fiddler to scale LLM and ML deployments, delivering high-performance AI, reducing costs, and ensuring responsible governance.

In this quick start, you'll learn how to:

1. **Connect** to Fiddler and set up your environment
2. **Create Projects & Applications** to organize your evaluations
3. **Build Datasets** with test cases for evaluation
4. **Use Built-in Evaluators** like Answer Relevance, Sentiment Analysis, and Coherence
5. **Create Custom Evaluators** for your specific needs
6. **Run Experiments** to evaluate your AI applications
7. **Analyze Results** and track performance over time

## Getting Started

0. Add credentials for AI providers in Fiddler's LLM Gateway tab.
1. Connect to Fiddler
2. Create a Project & Application
3. Build Datasets
4. Use Built-in Evaluators
5. Create Custom Evaluators
6. Run Experiments
7. Analyze Results

## 0. Imports and Configuration

In [ ]:
# Install the Fiddler Evaluations SDK
%pip install -q --upgrade fiddler-evals

# Core imports
import sys
from datetime import datetime

# Data handling
import pandas as pd

# Fiddler Evaluations SDK
from fiddler_evals import (
    __version__,
    init,
    Project,
    Application,
    Dataset,
    Experiment,
    evaluate,
    ScoreStatus,
    ExperimentItemStatus,
)
from fiddler_evals.pydantic_models.dataset import NewDatasetItem
from fiddler_evals.evaluators import (
    AnswerRelevance,
    Coherence,
    Conciseness,
    Sentiment,
)
from fiddler_evals.evaluators.base import Evaluator
from fiddler_evals.pydantic_models.score import Score

print(f"Running Fiddler Evals SDK version {__version__}")

## 1. Connect to Fiddler

Before you can start evaluating your AI applications, you'll need to connect to your Fiddler instance using the Evaluations SDK.

**What you need to get started:**
1. **Fiddler URL** - Your Fiddler instance URL (e.g., `https://your-org.fiddler.ai`)
2. **Authorization Token** - Found in the **Credentials** tab on your Fiddler **Settings** page

The connection establishes authentication and validates compatibility between your SDK version and the Fiddler server.

In [ ]:
# Replace with your Fiddler instance details
URL = ""  # Make sure to include the full URL (including https:// e.g. 'https://your_company_name.fiddler.ai')
TOKEN = ""  # Your Fiddler API token from Settings > Credentials

LLM_CREDENTIAL_NAME = ""  # The name of the LLM credential from Settings > LLM Gateway
LLM_MODEL_NAME = ""  # e.g., openai/gpt-4.1-nano. The name of the LLM model from Settings > LLM Gateway
FOUNDATIONAL_LLM_CONFIG = {
    "model": LLM_MODEL_NAME,
    "credential": LLM_CREDENTIAL_NAME,
}

**Configuration for this example** - customize these for your own use case:

In [ ]:
# Project, Application and Dataset names
PROJECT_NAME = "eval_demos"
APPLICATION_NAME = "llm_app_evaluation"
DATASET_NAME = "qa_evaluation_dataset"

Now let's establish the connection to your Fiddler instance:

In [ ]:
# Initialize connection to Fiddler
# The init function establishes authentication and validates server compatibility
init(url=URL, token=TOKEN)

print("✅ Successfully connected to Fiddler!")

## 2. Create a Project and Application

In Fiddler Evals, we organize evaluations using a hierarchical structure:
- **Projects** contain multiple applications and provide organizational boundaries
- **Applications** represent specific AI systems (chatbots, agents, etc.) you want to evaluate
- **Datasets** contain test cases used for evaluation
- **Experiments** track individual evaluation runs

Let's create our organizational structure:

In [ ]:
# Create or get the project
project = Project.get_or_create(name=PROJECT_NAME)
print(f"✅ Project: {project.name} (ID: {project.id})")

In [ ]:
# Create or get the application within the project
application = Application.get_or_create(
    name=APPLICATION_NAME,
    project_id=project.id,
)
print(f"✅ Application: {application.name} (ID: {application.id})")

## 3. Create a Dataset with Test Cases

Datasets in Fiddler Evals contain the test cases you'll use to evaluate your AI applications. Each test case typically includes:
- **Inputs**: The data you'll pass to your AI application (questions, prompts, etc.)
- **Expected Outputs**: What you expect the application to return
- **Metadata**: Additional information for categorization and filtering

Let's create a dataset with sample Q&A test cases:

In [ ]:
# Create or get the dataset
dataset = Dataset.get_or_create(
    name=DATASET_NAME,
    application_id=application.id,
    description="Q&A test cases for LLM evaluation",
)
print(f"✅ Dataset: {dataset.name} (ID: {dataset.id})")

In [ ]:
# Add test cases to the dataset
test_cases = [
    NewDatasetItem(
        inputs={"question": "What happens to you if you eat watermelon seeds?"},
        expected_outputs={
            "answer": "The watermelon seeds pass through your digestive system"
        },
        metadata={"type": "Adversarial", "category": "Misconceptions"},
    ),
    NewDatasetItem(
        inputs={"question": "What is the capital of France?"},
        expected_outputs={"answer": "Paris is the capital of France"},
        metadata={"type": "Factual", "category": "Geography"},
    ),
    NewDatasetItem(
        inputs={"question": "Who was the first person to walk on the moon?"},
        expected_outputs={
            "answer": "Neil Armstrong was the first person to walk on the moon"
        },
        metadata={"type": "Factual", "category": "History"},
    ),
    NewDatasetItem(
        inputs={"question": "What is the speed of light?"},
        expected_outputs={
            "answer": "The speed of light is approximately 299,792,458 meters per second"
        },
        metadata={"type": "Scientific", "category": "Physics"},
    ),
    NewDatasetItem(
        inputs={"question": "How do you make a simple omelet?"},
        expected_outputs={
            "answer": "Beat eggs, heat butter in a pan, pour eggs, cook until set, and fold in half"
        },
        metadata={"type": "Instructional", "category": "Cooking"},
    ),
]

# Check if dataset is empty before inserting
if not list(dataset.get_items()):
    print("\n📝 Adding test cases to dataset...")
    dataset.insert(test_cases)
    print(f"✅ Added {len(test_cases)} test cases")
else:
    print("\n📝 Test cases already present in the dataset")

## 4. Explore Built-in Evaluators

Fiddler Evals provides a comprehensive set of built-in evaluators for common AI evaluation tasks. Let's explore some of the key evaluators:

### 📊 Available Evaluators:
- **Answer Relevance**: Checks if the response addresses the question
- **Coherence**: Evaluates logical flow and consistency
- **Conciseness**: Measures response brevity and clarity
- **Sentiment**: Analyzes emotional tone
- **Regex Evaluators**: Pattern matching for specific formats

Let's see these evaluators in action!

In [ ]:
# Sample data for testing
sample_question = "What is the capital of France?"
good_answer = "Paris is the capital and largest city of France."
bad_answer = "Pizza is delicious and I love Italian food."

print("🧪 Testing Individual Evaluators")
print(f"Question: {sample_question}")
print(f"Good Answer: {good_answer}")
print(f"Bad Answer: {bad_answer}")
print("\n" + "=" * 80)

# Test Answer Relevance
relevance_evaluator = AnswerRelevance(**FOUNDATIONAL_LLM_CONFIG)
relevant_score = relevance_evaluator.score(prompt=sample_question, response=good_answer)
irrelevant_score = relevance_evaluator.score(
    prompt=sample_question, response=bad_answer
)

print("\n📊 Answer Relevance Results:")
print(f"Good Answer Score: {relevant_score.value} - {relevant_score.reasoning}")
print(f"Bad Answer Score: {irrelevant_score.value} - {irrelevant_score.reasoning}")

# Test Conciseness
conciseness_evaluator = Conciseness(**FOUNDATIONAL_LLM_CONFIG)
concise_response = "Paris is the capital and largest city of France."
verbose_response = "Paris is the capital and largest city of France. It is located in the north-central part of the country along the Seine River. Paris is known for its rich history, beautiful architecture, world-class museums like the Louvre, and iconic landmarks such as the Eiffel Tower and Notre-Dame Cathedral."

concise_score = conciseness_evaluator.score(response=concise_response)
verbose_score = conciseness_evaluator.score(response=verbose_response)

print("\n📊 Conciseness Results:")
print(f"Concise Answer Score: {concise_score.value} - {concise_score.reasoning}")
print(f"Verbose Answer Score: {verbose_score.value} - {verbose_score.reasoning}")

# Test Coherence
coherence_evaluator = Coherence(**FOUNDATIONAL_LLM_CONFIG)
coherent_score = coherence_evaluator.score(response=good_answer, prompt=sample_question)
incoherent_score = coherence_evaluator.score(
    response=bad_answer, prompt=sample_question
)

print("\n📊 Coherence Results:")
print(f"Coherent Answer Score: {coherent_score.value} - {coherent_score.reasoning}")
print(
    f"Incoherent Answer Score: {incoherent_score.value} - {incoherent_score.reasoning}"
)

# Test Sentiment
sentiment_evaluator = Sentiment()
positive_text = "I love this service! It's amazing and helpful."
negative_text = "This is terrible and I hate it completely."

positive_sentiment = sentiment_evaluator.score(positive_text)
negative_sentiment = sentiment_evaluator.score(negative_text)

print("\n😊 Sentiment Analysis Results:")
print(
    f"Positive text: {[x.label for x in positive_sentiment if x.label]} ({[x.value for x in positive_sentiment if x.value]})"
)
print(
    f"Negative text: {[x.label for x in negative_sentiment if x.label]} ({[x.value for x in negative_sentiment if x.value]})"
)

## 5. Create a Custom Evaluator

Sometimes you need evaluation logic specific to your use case. Fiddler Evals makes it easy to create custom evaluators by inheriting from the `Evaluator` base class.

Let's create a custom evaluator that checks if an answer is approximately the right length:

In [ ]:
class LengthEvaluator(Evaluator):
    """
    Custom evaluator that checks if a response length is appropriate.
    Gives higher scores for responses that are neither too short nor too long.

    This demonstrates how to create domain-specific evaluators for your use case.
    For example, you might want answers to be detailed but not verbose.

    Args:
        min_length: Minimum acceptable character count (default: 10)
                   Values below this indicate insufficient detail (e.g., "Yes" or "No")
        max_length: Maximum acceptable character count (default: 200)
                   Values above this indicate excessive verbosity
    """

    def __init__(self, min_length: int = 10, max_length: int = 200):
        super().__init__()
        self.min_length = min_length
        self.max_length = max_length

    def score(self, output: str) -> Score:
        """Score based on response length appropriateness."""
        length = len(output.strip())

        if length < self.min_length:
            score_value = 0.0
            reasoning = (
                f"Response too short ({length} chars, minimum {self.min_length})"
            )
        elif length > self.max_length:
            score_value = 0.5
            reasoning = f"Response too long ({length} chars, maximum {self.max_length})"
        else:
            score_value = 1.0
            reasoning = f"Response length appropriate ({length} chars)"

        return Score(
            name="length_check",
            evaluator_name=self.name,
            value=score_value,
            reasoning=reasoning,
        )


# Test our custom evaluator
length_evaluator = LengthEvaluator(min_length=15, max_length=100)

short_answer = "Yes"
good_answer = "Paris is the capital and largest city of France."
long_answer = "Paris is the capital and largest city of France. It is located in the north-central part of the country along the Seine River. Paris is known for its rich history, beautiful architecture, and world-class museums."

print("🔧 Testing Custom Length Evaluator:")
print(
    f"Short answer score: {length_evaluator.score(short_answer).value} - {length_evaluator.score(short_answer).reasoning}"
)
print(
    f"Good answer score: {length_evaluator.score(good_answer).value} - {length_evaluator.score(good_answer).reasoning}"
)
print(
    f"Long answer score: {length_evaluator.score(long_answer).value} - {length_evaluator.score(long_answer).reasoning}"
)

## 6. Run Your First Experiment

Now comes the exciting part - running a complete evaluation experiment! We'll create an evaluation task that simulates your LLM application and then evaluate it using multiple evaluators.

The `evaluate()` function orchestrates the entire process:
1. **Runs your evaluation task** on each dataset item
2. **Executes all evaluators** on the results
3. **Tracks the experiment** in Fiddler
4. **Returns comprehensive results** with scores and timing

Let's set up and run our experiment:

### Create a Mock LLM Function

For this demonstration, we'll create a simple mock LLM function. In practice, this would be your actual LLM API call.

In [ ]:
def mock_llm_response(question: str) -> str:
    """
    Mock LLM function that simulates an AI model's response.
    In practice, this would be your actual LLM API call.

    Note: Some responses below are intentionally incorrect to demonstrate
    how Fiddler's evaluators detect quality issues in LLM outputs.
    """
    # Simple responses for demonstration
    # NOTE: Some answers are intentionally wrong to show evaluator detection
    responses = {
        "What happens to you if you eat watermelon seeds?": "The watermelon seeds pass through your digestive system without harm.",
        "What is the capital of France?": "Grenoble is the capital and largest city of France.",  # Intentionally WRONG - should be Paris
        "Who was the first person to walk on the moon?": "Neil Armstrong was the first person to walk on the moon in 1969.",
        "What is the speed of light?": "The speed of light is approximately 299,792,458 miles per second in a vacuum.",  # Intentionally uses miles instead of meters
        "How do you make a simple omelet?": "Beat 2-3 eggs, heat butter in a pan, pour eggs in, cook until set, and fold in half. Season with salt and pepper.",
    }

    # Return matching response or a generic one
    return responses.get(
        question, f"I don't have specific information about: {question}"
    )

### Create a Mock LLM Task

For this demonstration, we'll create a simple mock LLM application.

In [ ]:
# Define our evaluation task function
def llm_eval_task(inputs: dict, extras: dict, metadata: dict) -> dict:
    """
    This function represents your AI application that you want to evaluate.
    It receives test case inputs and should return the outputs to be evaluated.

    Args:
        inputs: The input data from the dataset (e.g., {"question": "..."})
        extras: Additional context data (e.g., {"context": "..."})
        metadata: Any metadata associated with the test case

    Returns:
        dict: The outputs from your AI application (e.g., {"answer": "..."})
    """
    question = inputs.get("question", "")

    # In practice, this would be your actual LLM API call
    answer = mock_llm_response(question)

    return {"answer": answer}

### Select the Evaluators

Establish the individual evaluators to use in this evaluation experiment.

In [ ]:
# Set up our evaluators for the experiment
evaluators = [
    AnswerRelevance(**FOUNDATIONAL_LLM_CONFIG),  # Check if answer addresses question
    Conciseness(**FOUNDATIONAL_LLM_CONFIG),  # Check response brevity
    Coherence(**FOUNDATIONAL_LLM_CONFIG),  # Check logical flow
    Sentiment(),  # Analyze sentiment
    length_evaluator,  # Our custom length evaluator
]

print("🚀 Setting up experiment with evaluators:")
for evaluator in evaluators:
    print(f"  • {evaluator.name}")

print(f"\n📊 Dataset: {dataset.name} ({len(test_cases)} test cases)")
print("🎯 Task: LLM Q&A evaluation")
print("⏳ Starting experiment...")

### Run the Evaluation and View Experiment Results

In [ ]:
# Run the evaluation experiment!
experiment_result = evaluate(
    dataset=dataset,
    task=llm_eval_task,
    evaluators=evaluators,
    name_prefix="eval_demo",
    description="Comprehensive evaluation of LLM Q&A responses",
    metadata={
        "model_type": "mock_llm",
        "evaluation_version": "v1.1",
        "evaluator_count": len(evaluators),
    },
    # Map evaluator parameters to task outputs
    score_fn_kwargs_mapping={
        "question": lambda x: x["inputs"]["question"],  # Map 'question' to question
        "response": "answer",  # Map 'response' to answer
        "output": "answer",  # Map 'output' to answer
        "text": "answer",  # Map 'text' to answer
        "prompt": lambda x: x["inputs"]["question"],  # Map 'prompt' to question
    },
    max_workers=2,  # Process 2 test cases concurrently
)

print("\n✅ Experiment completed!")
print(f"📊 Evaluated {len(experiment_result.results)} test cases")
print(f"🧪 Used {len(evaluators)} evaluators")
print(
    f"📈 Generated {sum(len(result.scores) for result in experiment_result.results)} total scores"
)

## 8. Analyze Experiment Results (Programmatic Access)

Now let's dive into the results! Fiddler Evals provides comprehensive result tracking with detailed scores, timing information, and error handling.

Let's explore what we got from our experiment:

In [ ]:
# Detailed analysis of experiment results
print("🔍 Detailed Results Analysis")
print("=" * 80)

for i, result in enumerate(experiment_result.results):
    item = result.experiment_item
    scores = result.scores

    print(f"\n📝 Test Case {i + 1}:")
    print(f"   Dataset Item ID: {item.dataset_item_id}")
    print(f"   Status: {item.status}")
    print(f"   Execution Time: {item.duration_ms}ms")

    if item.status == ExperimentItemStatus.SUCCESS:
        answer = item.outputs.get("answer", "N/A")
        print(f"   Answer: {answer[:100]}{'...' if len(answer) > 100 else ''}")

        # Show all scores for this test case
        print(f"   Scores ({len(scores)}):")
        for score in scores:
            status_emoji = "✅" if score.status == ScoreStatus.SUCCESS else "❌"
            score_value = score.value if score.value is not None else score.label
            print(f"     {status_emoji} {score.name}: {score_value}")
            if score.reasoning:
                print(f"        Reasoning: {score.reasoning}")

## 9. Export Results for Further Analysis

You can easily export your experiment results for further analysis, reporting, or integration with other tools:

In [ ]:
# Convert results to DataFrame for easy analysis
results_data = []

for result in experiment_result.results:
    item = result.experiment_item

    # Base row data
    row = {
        "dataset_item_id": item.dataset_item_id,
        "status": item.status,
        "duration_ms": item.duration_ms,
        "answer": item.outputs.get("answer", "") if item.outputs else "",
    }

    # Add scores as columns
    for score in result.scores:
        row[f"{score.name}_score"] = score.value
        row[f"{score.name}_reasoning"] = score.reasoning
        row[f"{score.name}_status"] = score.status

    results_data.append(row)

# Create DataFrame
results_df = pd.DataFrame(results_data)

print("📊 Results DataFrame created!")
print(f"Shape: {results_df.shape}")
print("\nColumns:")
for col in results_df.columns:
    print(f"  • {col}")

# Display first few rows
print("\n📋 Sample Results:")
print(results_df.head())

# Save to CSV for further analysis
csv_filename = f"experiment_results_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
results_df.to_csv(csv_filename, index=False)
print(f"\n💾 Results saved to: {csv_filename}")

## 7. View Results in Fiddler UI

Your experiment has been saved to Fiddler! Now let's see how to view and analyze your results in the Fiddler web interface.

### 📊 Accessing Your Experiment

1. **Open Fiddler**: Navigate to your Fiddler instance (the URL you used to connect)
2. **Find Your Project**: Go to **Projects** in the left sidebar → Select `eval_demos`
3. **Open Your Application**: Click on the `llm_app_evaluation` application
4. **View Experiments**: Click the **Experiments** tab to see all evaluation runs
5. **Select Your Experiment**: Find the experiment that starts with `eval_demo_` (sorted by most recent)

### 🔍 What You'll See in the UI

The Fiddler UI provides a comprehensive view of your evaluation results:

- **Summary Metrics**: Aggregate scores across all evaluators
- **Test Case Details**: Individual results for each question/answer pair
- **Evaluator Scores**: Detailed breakdown of each evaluator's assessment
- **Performance Metrics**: Execution time and status for each test case
- **Filtering & Search**: Filter by metadata (type, category) to find specific test cases
- **Export Options**: Download results as CSV or JSON for further analysis

### 💡 Next Steps in the UI

- **Compare Experiments**: Select multiple experiments to see side-by-side comparisons
- **Analyze Patterns**: Look for categories with consistently low scores
- **Track Progress**: Run the same evaluation after making improvements to see score changes
- **Share Results**: Export reports or share experiment URLs with your team

The experiment data displayed below matches what you'll see in the Fiddler UI!

---

# Advanced Topics

The following sections cover advanced usage patterns. These are optional for getting started but useful as you scale your evaluation workflows.

## 10. Troubleshooting & Diagnostics

Run this diagnostic cell if you encounter any issues:

In [ ]:
# Diagnostic checks
print("🔧 System Diagnostics\n")
print("=" * 80)

# Check Python version
print(f"\n✓ Python Version: {sys.version.split()[0]}")

# Check SDK version
print(f"✓ Fiddler Evals SDK Version: {__version__}")

# Check connection
print(f"✓ Connected to: {URL}")

# Check project structure
print("\n📁 Project Structure:")
print(f"  └─ Project: {project.name} ({project.id})")
print(f"     └─ Application: {application.name} ({application.id})")
print(f"        └─ Dataset: {dataset.name} ({dataset.id})")

# Check evaluators configured
print(f"\n🧪 Evaluators Configured: {len(evaluators)}")
for evaluator in evaluators:
    print(f"  • {evaluator.name}")

# Check dataset items
dataset_items = list(dataset.get_items())
print(f"\n📊 Dataset: {len(dataset_items)} test cases")

# Check recent experiments
try:
    experiments = list(Experiment.list(application_id=application.id))
    print(f"\n🔬 Total Experiments: {len(experiments)}")
    if experiments:
        print(f"   Most Recent:")
        for exp in experiments[:3]:
            print(f"   • {exp.name} - Status: {exp.status}")
except Exception as e:
    print(f"\n⚠️ Could not retrieve experiments: {e}")

print("\n" + "=" * 80)
print("✅ Diagnostics Complete")

## 🎉 Congratulations!

You've successfully completed the Fiddler Evaluations SDK quickstart! Here's what you accomplished:

✅ **Connected** to Fiddler and set up your environment  
✅ **Created** a project and application structure  
✅ **Built** a dataset with test cases  
✅ **Used** built-in evaluators (Answer Relevance, Sentiment, etc.)  
✅ **Created** a custom evaluator for your specific needs  
✅ **Ran** a complete evaluation experiment  
✅ **Analyzed** results with detailed metrics and insights  
✅ **Exported** data for further analysis  

## 🚀 What's Next?

Now that you've mastered the basics, here are some next steps:

### 📚 **Learn More:**
- [Getting Started with Fiddler Evals](https://docs.fiddler.ai/first-steps/getting-started-with-fiddler-evals)
- [Evals SDK Technical Reference](https://docs.fiddler.ai/technical-reference/fiddler-evals-sdk)
- [Evals SDK Advanced Guide](https://github.com/fiddler-labs/fiddler-examples/blob/main/quickstart/latest/Fiddler_Quickstart_Advanced_Evaluations_SDK.ipynb)

### 🛠️ **Build Your Own:**
- **Replace** the mock LLM with your actual model API
- **Add** your own test cases and evaluation criteria
- **Create** custom evaluators for domain-specific requirements
- **Set up** automated evaluation pipelines

### 🏭 **Production Usage:**
- **Integrate** evaluations into your CI/CD pipeline
- **Monitor** model performance over time
- **Compare** different model versions
- **Scale** to larger datasets with parallel processing

**Happy Evaluating!** 🎯

---

**Questions?**  
  
Check out [our docs](https://docs.fiddler.ai/) for a more detailed explanation of what Fiddler has to offer.